In [ ]:
import scipy.io as io
import numpy as np
import os
import torch
import time

In [ ]:
# data shape is [repeat, frame, cell]
# output is [N, cellNum*stride]
def split_data_FCAE_original(data, stride=6):
    repeatNum, frameNum, cellNum = data.shape
    result = np.empty([0, cellNum*stride])
    for i in range(repeatNum):
        for j in range(frameNum - stride):
            result = np.concatenate((result, data[i, j:j+stride, :].reshape((1, cellNum*stride))))
    return result

def split_data_FCAE(data, stride=6, withoutGray=False, vedio='PDG'):
    data = torch.tensor(data)
    if withoutGray:
        if vedio == 'PDG':
            data = data.reshape(8, 12, 60, -1) 
            data = data[:, :, :20].reshape((8, 20*12, -1))
        elif vedio == 'MOV':
            data = data[:, 50:]
    repeat, frame, cell = data.shape
    if stride > frame:
        print("stride cannot larger than frame number")
        return None
    repeat_data = data.repeat((1, 1, stride))
    reshape_data = repeat_data.reshape((repeat, frame, stride, cell))
    for i in range(1, stride):
        reshape_data[:, :-i, i] = reshape_data[:, i:, i]
    reshape_data = reshape_data.reshape((repeat*frame, cell*stride))
    return reshape_data.numpy()

# data shape is [repeat, frame, cell]
# output is [N, stride, cellNum]
def split_data_LSTMAE_original(data, stride=6):
    repeatNum, frameNum, cellNum = data.shape
    result = np.empty([0, stride, cellNum])
    for i in range(repeatNum):
        for j in range(frameNum-stride):
            result = np.concatenate((result, data[i:i+1, j:j+stride, :]))
    
    return result

def split_data_LSTMAE(data, stride=6, withoutGray=False, vedio='PDG'):
    data = torch.tensor(data)
    if withoutGray:
        if vedio == 'PDG':
            data = data.reshape(8, 12, 60, -1) 
            data = data[:, :, :20].reshape((8, 20*12, -1))
        elif vedio == 'MOV':
            data = data[:, 50:]
    repeat, frame, cell = data.shape
    if stride > frame:
        print("stride cannot larger than frame number")
        return None
    repeat_data = data.repeat((1, 1, stride))
    reshape_data = repeat_data.reshape((repeat, frame, stride, cell))
    for i in range(1, stride):
        reshape_data[:, :-i, i] = reshape_data[:, i:, i]
    reshape_data = reshape_data.reshape((repeat*frame, stride, cell))
    return reshape_data.numpy()


In [ ]:
# 测试预处理函数
d = [i for i in range(3*10*5)]
t = torch.as_tensor(d).reshape(3, 10, 5)
print(t)
result = split_data_LSTMAE(t, stride=2)
# print(result)


In [ ]:
data = io.loadmat('./Main experiments/Mouse 2/Window 1/RespData.mat')
# data = io.loadmat('./Main experiments/Mouse 1/RespData.mat')
MOV_data = data['RespData'][0][0][1][0][0][0][:, :, :, 0]
PDG_data = data['RespData'][0][0][0][0][0][0][:, :, :, 0]

# tic = time.time()
# original_result = split_data_LSTMAE_original(MOV_data)
# toc = time.time()
# print('orignal algorithm cost %d' % (toc-tic))

print(PDG_data.shape)

tic = time.time()
new_result = split_data_LSTMAE(PDG_data, stride=12, withoutGray=True, vedio='PDG')
toc = time.time()
print('new algorithm cost: %d' % (toc-tic))
print(new_result.shape)

In [ ]:
def standardize(data):
    repeatNum, frameNum, cellNum = data.shape
    data = data.reshape((repeatNum * frameNum, cellNum))
    data = (data - np.mean(data, axis=1, keepdims=True)) / np.std(data, axis=1, keepdims=True)
    data = data.reshape((repeatNum, frameNum, cellNum))
    return data

In [ ]:
filePath = './Main experiments/Mouse {index}/RespData.mat'
mouseNum = 1
savePDGPath = './myData/Day{day}_PDG/PDG_mouse{index}_{model}.npy'
saveMOVPath = './myData/Day{day}_MOV/MOV_mouse{index}_{model}.npy'
stride = 6
withoutgray = True

In [ ]:
for i in range(mouseNum):
    if i+1 != 2:
        data = io.loadmat(filePath.format(index=i+1))
    else:
        data = io.loadmat('./Main experiments/Mouse 2/Window 1/RespData.mat')

    sessionNum = data['RespData'][0][0][0][0][0][0].shape[3]

    for j in range(0, sessionNum):
        PDG_data = data['RespData'][0][0][0][0][0][0][:, :, :, j]
        MOV_data = data['RespData'][0][0][1][0][0][0][:, :, :, j]

        if not os.path.isdir('./myData/Day{index}_MOV'.format(index=j*7)):
            os.mkdir('./myData/Day{day}_MOV/'.format(day=j*7))
            os.mkdir('./myData/Day{day}_PDG/'.format(day=j*7))

        repeatNumMOV, frameNumMOV, cellNumMOV = MOV_data.shape
        MOV_data = standardize(MOV_data)

        MOV_data_split_LSTMAE = split_data_LSTMAE(MOV_data, withoutGray=True, vedio='MOV')
        np.save(saveMOVPath.format(day=j*7, index=i+1, model='LSTMAE'), MOV_data_split_LSTMAE)
        MOV_data_split_FCAE = split_data_FCAE(MOV_data, withoutGray=True, vedio='MOV')
        np.save(saveMOVPath.format(day=j*7, index=i+1, model='FCAE'), MOV_data_split_FCAE)
        print(f'save success mouse{i+1} day{j*7} MOV')

        repeatNumPDG, frameNumPDG, cellNumPDG = PDG_data.shape
        PDG_data = standardize(PDG_data)

        PDG_data_split_LSTMAE = split_data_LSTMAE(PDG_data, withoutGray=True, vedio='PDG')
        np.save(savePDGPath.format(day=j*7, index=i+1, model='LSTMAE'), PDG_data_split_LSTMAE)
        PDG_data_split_FCAE = split_data_FCAE(PDG_data, withoutGray=True, vedio='PDG')
        np.save(savePDGPath.format(day=j*7, index=i+1, model='FCAE'), PDG_data_split_FCAE)
        print(f'save success mouse{i+1} day{j*7} PDG')
